# 03: Descriptive Analysis

This notebook presents descriptive statistics on sycophancy in the LMSYS dataset.

## Research Questions
1. How prevalent is sycophancy across models?
2. Does sycophancy correlate with winning battles?
3. How does sycophancy vary by topic domain?

In [ ]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root / 'src'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

## 1. Load and Merge Data

In [ ]:
from quant_syco.data.process import build_battle_table
from quant_syco.features.topics import compute_topic_features
from quant_syco.features.lexical import compute_response_length_features
from quant_syco.config import LABELS_DIR

# Load battles
battles = build_battle_table()
battles = compute_topic_features(battles)
battles = compute_response_length_features(battles)

# Load labels
label_files = list(LABELS_DIR.glob('labels_*_merged.parquet'))
if label_files:
    labels = pd.read_parquet(label_files[0])
    df = battles.merge(labels, on='question_id', how='left')
else:
    raise FileNotFoundError("Run 'make label' first")

print(f"Dataset: {len(df):,} battles with labels")
df.head()

## 2. Summary Statistics

In [ ]:
from quant_syco.analysis.descriptive import compute_summary_statistics

summary = compute_summary_statistics(df)
summary

In [ ]:
# Distribution comparison: A vs B
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for i, metric in enumerate(['sycophancy', 'politeness']):
    a_col = f'{metric}_a'
    b_col = f'{metric}_b'
    
    if a_col in df.columns and b_col in df.columns:
        a_mean = df[a_col].mean()
        b_mean = df[b_col].mean()
        
        x = np.arange(4)
        width = 0.35
        
        a_counts = df[a_col].value_counts(normalize=True).reindex([0,1,2,3], fill_value=0)
        b_counts = df[b_col].value_counts(normalize=True).reindex([0,1,2,3], fill_value=0)
        
        axes[i].bar(x - width/2, a_counts.values, width, label=f'A (μ={a_mean:.2f})')
        axes[i].bar(x + width/2, b_counts.values, width, label=f'B (μ={b_mean:.2f})')
        axes[i].set_xlabel('Score')
        axes[i].set_ylabel('Proportion')
        axes[i].set_title(f'{metric.title()} Distribution')
        axes[i].set_xticks(x)
        axes[i].legend()

plt.tight_layout()

## 3. Sycophancy by Model

In [ ]:
from quant_syco.analysis.descriptive import compute_sycophancy_by_model

by_model = compute_sycophancy_by_model(df, 'model_a', 'sycophancy_a')
by_model_filtered = by_model[by_model['n_samples'] >= 50]  # Minimum sample size

print(f"Models with ≥50 samples: {len(by_model_filtered)}")
by_model_filtered.head(15)

In [ ]:
# Visualize top/bottom models
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

top_10 = by_model_filtered.head(10)
bottom_10 = by_model_filtered.tail(10)

axes[0].barh(top_10['model_a'], top_10['mean_sycophancy'], color='coral')
axes[0].set_xlabel('Mean Sycophancy Score')
axes[0].set_title('Top 10 Most Sycophantic Models')
axes[0].invert_yaxis()

axes[1].barh(bottom_10['model_a'], bottom_10['mean_sycophancy'], color='steelblue')
axes[1].set_xlabel('Mean Sycophancy Score')
axes[1].set_title('Top 10 Least Sycophantic Models')
axes[1].invert_yaxis()

plt.tight_layout()

## 4. Sycophancy and Battle Outcomes

In [ ]:
from quant_syco.analysis.descriptive import (
    compute_win_rate_by_sycophancy,
    compute_sycophancy_differential_effect
)

# Win rate by sycophancy level
win_by_syco = compute_win_rate_by_sycophancy(df, 'sycophancy_a', 'winner')
print("Win Rate by Sycophancy Level (Side A):")
win_by_syco

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

ax.bar(win_by_syco['sycophancy_a'], win_by_syco['win_rate'], 
       yerr=np.sqrt(win_by_syco['win_rate'] * (1 - win_by_syco['win_rate']) / win_by_syco['n']),
       capsize=5, color='steelblue')
ax.axhline(y=0.5, color='red', linestyle='--', label='Chance level')
ax.set_xlabel('Sycophancy Score')
ax.set_ylabel('Win Rate')
ax.set_title('Win Rate by Sycophancy Level')
ax.legend()
plt.tight_layout()

In [ ]:
# Differential effect: does MORE sycophantic response win?
diff_effect = compute_sycophancy_differential_effect(df)
print("\nWin Rate by Sycophancy Differential (A - B):")
diff_effect

## 5. Sycophancy by Topic Domain

In [ ]:
# Mean sycophancy by topic
topic_stats = df.groupby('topic').agg({
    'sycophancy_a': ['mean', 'std', 'count'],
    'politeness_a': ['mean'],
}).round(3)
topic_stats.columns = ['syco_mean', 'syco_std', 'n', 'polite_mean']
topic_stats = topic_stats.sort_values('syco_mean', ascending=False)
topic_stats

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

x = np.arange(len(topic_stats))
width = 0.35

ax.bar(x - width/2, topic_stats['syco_mean'], width, label='Sycophancy', color='coral')
ax.bar(x + width/2, topic_stats['polite_mean'], width, label='Politeness', color='steelblue')

ax.set_xticks(x)
ax.set_xticklabels(topic_stats.index, rotation=45, ha='right')
ax.set_ylabel('Mean Score')
ax.set_title('Sycophancy and Politeness by Topic')
ax.legend()
plt.tight_layout()

## 6. Correlations

In [ ]:
from quant_syco.analysis.descriptive import compute_correlations

# Correlation matrix
numeric_cols = ['sycophancy_a', 'sycophancy_b', 'politeness_a', 'politeness_b',
                'assistant_a_word_count', 'assistant_b_word_count']
numeric_cols = [c for c in numeric_cols if c in df.columns]

corr_matrix = df[numeric_cols].corr(method='spearman')

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdBu_r', center=0, ax=ax)
ax.set_title('Spearman Correlation Matrix')
plt.tight_layout()

## 7. Key Findings Summary

In [ ]:
print("KEY DESCRIPTIVE FINDINGS")
print("=" * 50)

# Mean sycophancy
mean_syco = df['sycophancy_a'].mean()
print(f"\n1. Mean sycophancy score: {mean_syco:.2f} (0-3 scale)")

# Percentage high sycophancy
high_syco_pct = (df['sycophancy_a'] >= 2).mean() * 100
print(f"2. Responses with moderate+ sycophancy (≥2): {high_syco_pct:.1f}%")

# Sycophancy-politeness correlation
if 'politeness_a' in df.columns:
    sp_corr = df[['sycophancy_a', 'politeness_a']].corr().iloc[0, 1]
    print(f"3. Sycophancy-politeness correlation: {sp_corr:.2f}")

# Win rate for high vs low sycophancy
high_syco = df[df['sycophancy_a'] >= 2]
low_syco = df[df['sycophancy_a'] <= 1]
high_win = (high_syco['winner'] == 'model_a').mean()
low_win = (low_syco['winner'] == 'model_a').mean()
print(f"4. Win rate for high sycophancy (≥2): {high_win:.1%}")
print(f"   Win rate for low sycophancy (≤1): {low_win:.1%}")

# Most sycophantic model
if len(by_model_filtered) > 0:
    top_model = by_model_filtered.iloc[0]
    print(f"5. Most sycophantic model: {top_model['model_a']} (μ={top_model['mean_sycophancy']:.2f})")

## Next Steps

Continue with `04_causal_modeling.ipynb` for causal inference analysis.